In [1]:
import numpy as np
import pandas as pd

from sklearn.preprocessing import LabelEncoder
from sklearn.cross_validation import StratifiedShuffleSplit

train = pd.read_csv('/home/tushar/extras/all/train.csv')
test = pd.read_csv('/home/tushar/extras/all/test.csv')

/home/tushar/miniconda3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)


In [2]:
train.shape
train.head()

,id,species,margin1,margin2,margin3,margin4,margin5,margin6,margin7,margin8,...,texture55,texture56,texture57,texture58,texture59,texture60,texture61,texture62,texture63,texture64
0,1,Acer_Opalus,0.007812,0.023438,0.023438,0.003906,0.011719,0.009766,0.027344,0.0,...,0.007812,0.000000,0.002930,0.002930,0.035156,0.0,0.0,0.004883,0.000000,0.025391
1,2,Pterocarya_Stenoptera,0.005859,0.000000,0.031250,0.015625,0.025391,0.001953,0.019531,0.0,...,0.000977,0.000000,0.000000,0.000977,0.023438,0.0,0.0,0.000977,0.039062,0.022461
2,3,Quercus_Hartwissiana,0.005859,0.009766,0.019531,0.007812,0.003906,0.005859,0.068359,0.0,...,0.154300,0.000000,0.005859,0.000977,0.007812,0.0,0.0,0.000000,0.020508,0.002930
3,5,Tilia_Tomentosa,0.000000,0.003906,0.023438,0.005859,0.021484,0.019531,0.023438,0.0,...,0.000000,0.000977,0.000000,0.000000,0.020508,0.0,0.0,0.017578,0.000000,0.047852
4,6,Quercus_Variabilis,0.005859,0.003906,0.048828,0.009766,0.013672,0.015625,0.005859,0.0,...,0.096680,0.000000,0.021484,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.031250


In [4]:
def encode(train, test):
    le = LabelEncoder().fit(train.species)
    labels = le.transform(train.species)
    
    classes = list(le.classes_)
    
    test_ids = test.id
    
    train = train.drop(['species', 'id'], axis=1)
    test = test.drop(['id'], axis=1)
    
    return train, labels, test, test_ids, classes

train, labels, test, test_ids, classes = encode(train, test)
train.head(1)

,margin1,margin2,margin3,margin4,margin5,margin6,margin7,margin8,margin9,margin10,...,texture55,texture56,texture57,texture58,texture59,texture60,texture61,texture62,texture63,texture64
0,0.007812,0.023438,0.023438,0.003906,0.011719,0.009766,0.027344,0.0,0.001953,0.033203,...,0.007812,0.0,0.00293,0.00293,0.035156,0.0,0.0,0.004883,0.0,0.025391


In [5]:
#Stratified Train/Test split
SSS = StratifiedShuffleSplit(labels, 10, test_size=0.2, random_state=23)

for train_index, test_index in SSS:
    X_train, X_test = train.values[train_index], train.values[test_index]
    y_train, y_test = labels[train_index], labels[test_index]


In [8]:
from sklearn.metrics import accuracy_score, log_loss
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC, NuSVC

In [12]:
classifiers = [
    KNeighborsClassifier(3),
    SVC(kernel='rbf', C=0.025, probability=True),
    NuSVC(probability=True),
    DecisionTreeClassifier(),
    LinearDiscriminantAnalysis()]

log_colms=["Classifier", "Accuracy", "Log Loss"]
log = pd.DataFrame(columns=log_colms)

for clf in classifiers:
    clf.fit(X_train, y_train)
    name = clf.__class__.__name__
    
    print('-'*20)
    print(name)
    
    print('**Results**')
    train_predictions = clf.predict(X_test)
    acc = accuracy_score(y_test, train_predictions)
    print("Accuracy: {:.4%}".format(acc))
    
    train_predictions = clf.predict_proba(X_test)
    ll = log_loss(y_test, train_predictions)
    print("Log Loss: {}".format(ll))
    
    log_entry = pd.DataFrame([[name, acc*100, ll]], columns=log_colms)
    log = log.append(log_entry)
    print("-"*20)

--------------------
KNeighborsClassifier
**Results**
Accuracy: 88.8889%
Log Loss: 1.5755075129933762
--------------------
--------------------
SVC
**Results**
Accuracy: 81.8182%
Log Loss: 4.60669678575522
--------------------
--------------------
NuSVC
**Results**
Accuracy: 88.3838%
Log Loss: 2.4898601555592235
--------------------
--------------------
DecisionTreeClassifier
**Results**
Accuracy: 64.1414%
Log Loss: 12.38511678807413
--------------------


/home/tushar/miniconda3/lib/python3.6/site-packages/sklearn/discriminant_analysis.py:442: UserWarning: The priors do not sum to 1. Renormalizing
  UserWarning)
/home/tushar/miniconda3/lib/python3.6/site-packages/sklearn/discriminant_analysis.py:388: UserWarning: Variables are collinear.
  warnings.warn("Variables are collinear.")


--------------------
LinearDiscriminantAnalysis
**Results**
Accuracy: 97.9798%
Log Loss: 0.9301977763139313
--------------------


/home/tushar/miniconda3/lib/python3.6/site-packages/sklearn/discriminant_analysis.py:510: RuntimeWarning: overflow encountered in exp
  np.exp(prob, prob)


The best accuracy score and log loss score was of Linear Discriminant Analysis

In [17]:
best_classifier = LinearDiscriminantAnalysis()
best_classifier.fit(X_train, y_train)
test_prediction = best_classifier.predict_proba(test)

submit = pd.DataFrame(test_prediction, columns=classes)
submit.insert(0,'id', test_ids)
submit.reset_index()

/home/tushar/miniconda3/lib/python3.6/site-packages/sklearn/discriminant_analysis.py:442: UserWarning: The priors do not sum to 1. Renormalizing
  UserWarning)
/home/tushar/miniconda3/lib/python3.6/site-packages/sklearn/discriminant_analysis.py:388: UserWarning: Variables are collinear.
  warnings.warn("Variables are collinear.")
/home/tushar/miniconda3/lib/python3.6/site-packages/sklearn/discriminant_analysis.py:510: RuntimeWarning: overflow encountered in exp
  np.exp(prob, prob)


,index,id,Acer_Capillipes,Acer_Circinatum,Acer_Mono,Acer_Opalus,Acer_Palmatum,Acer_Pictum,Acer_Platanoids,Acer_Rubrum,...,Salix_Fragilis,Salix_Intergra,Sorbus_Aria,Tilia_Oliveri,Tilia_Platyphyllos,Tilia_Tomentosa,Ulmus_Bergmanniana,Viburnum_Tinus,Viburnum_x_Rhytidophylloides,Zelkova_Serrata
0,0,4,1.072044e-72,3.122113e-141,2.528986e-119,1.563013e-10,0.000000e+00,5.464621e-77,1.876137e-107,1.985294e-92,...,4.924035e-185,1.103194e-60,1.375676e-120,2.666221e-76,1.665729e-82,6.261301e-54,4.220956e-120,1.303294e-70,8.203341e-81,7.223632e-100
1,1,7,7.299118e-84,2.139038e-164,2.414536e-93,5.823260e-16,0.000000e+00,7.511342e-53,1.866346e-21,5.013291e-63,...,3.902529e-174,4.634589e-74,3.150828e-138,2.074729e-71,4.713480e-101,3.961899e-39,4.218634e-106,6.737630e-35,1.018464e-148,1.605866e-70
2,2,9,1.250918e-57,2.500846e-01,1.458043e-98,6.179676e-25,1.369167e-39,2.602388e-70,1.092184e-112,6.503154e-23,...,5.057997e-105,4.908310e-70,4.898294e-138,3.672661e-57,2.154262e-45,3.034889e-08,1.473641e-06,1.163684e-86,1.818950e-176,6.185620e-20
3,3,12,5.016924e-58,1.428327e-01,9.987374e-155,1.345386e-27,1.879105e-244,4.594242e-117,5.088357e-105,1.047155e-41,...,1.126869e-142,2.778146e-99,3.230107e-104,6.933291e-59,2.522766e-46,4.890661e-18,1.428327e-01,1.446746e-80,1.353366e-165,1.428327e-01
4,4,13,2.679067e-24,1.230317e-58,4.188163e-113,1.119974e-43,4.365100e-292,5.512297e-95,6.847828e-67,3.001831e-31,...,1.108099e-67,4.496478e-81,3.553927e-84,4.118120e-47,2.043628e-08,1.428571e-01,1.428571e-01,9.961470e-58,7.176411e-136,8.352855e-50
5,5,16,4.446073e-58,1.051810e-143,6.359958e-107,1.000000e+00,0.000000e+00,7.520445e-82,1.447300e-117,1.226681e-70,...,4.276227e-138,8.595099e-58,1.262737e-111,5.397557e-72,4.392124e-47,2.252963e-25,1.125949e-101,1.419130e-34,2.498407e-98,4.699769e-52
6,6,19,3.255337e-47,5.104178e-131,9.365234e-91,1.000000e+00,0.000000e+00,2.803409e-87,1.218134e-116,3.839908e-79,...,3.684647e-147,3.304552e-56,1.096080e-98,4.292328e-65,2.724641e-67,1.405434e-22,6.083795e-104,4.625522e-45,3.940749e-114,6.963742e-49
7,7,23,1.614635e-95,2.085602e-141,2.586828e-81,2.685140e-23,0.000000e+00,5.358847e-14,2.892027e-85,3.877350e-83,...,2.759930e-158,7.724934e-55,1.892987e-121,4.375873e-68,1.705238e-70,5.151870e-58,1.403820e-131,2.476210e-27,8.762595e-118,4.173118e-61
8,8,24,6.849754e-50,5.188209e-138,3.124882e-69,3.930359e-27,0.000000e+00,5.157798e-70,9.160205e-94,4.236661e-71,...,3.492905e-140,1.731016e-79,3.048524e-122,3.637908e-51,2.000742e-95,7.991351e-26,1.470031e-118,5.611961e-47,5.150528e-76,1.286455e-85
9,9,28,3.388044e-30,3.387678e-62,3.356426e-105,1.607026e-21,1.975163e-280,6.008576e-59,1.562269e-56,4.371972e-07,...,2.234495e-88,5.072762e-76,2.394378e-91,1.697233e-42,4.334874e-21,4.999998e-01,1.205599e-20,7.127952e-92,3.489196e-155,1.145411e-49


Using Neural Network